<a href="https://colab.research.google.com/github/rowida2/recommendation-system-project/blob/main/scrao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

uploaded = files.upload()


Saving tourism_places (1).csv to tourism_places (1).csv
Saving users.csv to users.csv


In [4]:
uploaded = files.upload()

Saving interactions.csv to interactions (1).csv
Saving posts.csv to posts (1).csv


In [5]:

from pandas import read_csv


plac=read_csv("tourism_places (1).csv")
posts=read_csv("posts.csv")
interactions=read_csv("interactions.csv")
users=read_csv("users.csv")


In [6]:
posts.head()

,post_id,user_id,place_id,content
0,450,157,115,Extreme sports at Modern Quad Biking Area of S...
1,634,26,460,Push your limits at Modern Quad Biking Area of...
2,1,226,424,Cultural overload at Dahab Mosque - in the bes...
3,756,190,480,Amazing experience at Blue Citadel of Marsa Al...
4,46,140,122,Lounging at White Island of Sharm El Sheikh wi...


In [7]:
interactions.tail()

,interaction_id,user_id,place_id,interaction_type,rating,timestamp
4993,4994,250,263,like,4,2024-09-09
4994,4995,250,7,view,5,2025-09-24
4995,4996,250,302,like,5,2024-04-12
4996,4997,250,388,visit,5,2025-03-31
4997,4998,250,141,visit,4,2025-04-20


In [8]:
users.count()

,0
user_id,250
name,250
age,250
gender,250
interests,250
budget_preference,250
travel_style,250
visited_cities,218


In [9]:
plac.count()

,0
place_id,500
name,500
city,500
category,500
budget_level,500
suitable_for,500
tags,500


In [10]:
weights = {"view": 1, "like": 2, "save": 3,"visit": 4}

data = interactions.merge(plac, on="place_id")
data["score"] = data["interaction_type"].map(weights)

user_interest = (
    data
    .groupby(["user_id", "category"])["score"]
    .sum()
    .reset_index()
)
data


,interaction_id,user_id,place_id,interaction_type,rating,timestamp,name,city,category,budget_level,suitable_for,tags,score
0,1,1,328,visit,5,2025-01-18,Suez Medical Tourism Center,Suez,medical,high,all,"relaxation, healing, massage, mineral water",4
1,2,1,67,like,5,2024-02-14,Royal Rock Climbing Site of Fayoum,Fayoum,adventure,mid,solo,"hiking, wildlife, safari",2
2,3,1,68,save,4,2024-07-08,Giza Diving Center,Giza,beach,low,all,"swimming, beach, sunset",3
3,4,1,93,save,5,2024-09-07,Giza Quad Biking Area,Giza,adventure,mid,solo,"excitement, hiking, mountain, wildlife",3
4,5,1,122,visit,5,2025-06-26,Royal Monastery of Taba,Taba,historical,low,all,"roman, educational, museum",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,4994,250,263,like,4,2024-09-09,Ancient Safari Park of Port Said,Port Said,adventure,high,solo,"hiking, excitement, adventure, desert",2
4994,4995,250,7,view,5,2025-09-24,White Island of Alexandria,Alexandria,beach,high,family,"boat trip, diving, snorkeling, swimming",1
4995,4996,250,302,like,5,2024-04-12,Hurghada Archaeological Site,Hurghada,historical,low,all,"islamic, heritage, ancient",2
4996,4997,250,388,visit,5,2025-03-31,Cairo Wellness Center,Cairo,medical,high,all,"relaxation, massage, therapeutic",4


In [11]:
interactions["interaction_type"] = interactions["interaction_type"].str.lower().str.strip()


weights = {
    "view": 1,
    "like": 2,
    "save": 3,
    "visit": 4
}

# حساب score
interactions["score"] = interactions["interaction_type"].map(weights)





In [12]:
data["interaction_type"].unique

<bound method Series.unique of 0       visit
1        like
2        save
3        save
4       visit
        ...  
4993     like
4994     view
4995     like
4996    visit
4997    visit
Name: interaction_type, Length: 4998, dtype: object>

In [13]:
data = interactions.merge(plac[["place_id", "category", "budget_level", "suitable_for"]], on="place_id", how="left")


In [14]:
user_interest = (
    data.groupby(["user_id", "category"])["score"]
    .sum()
    .reset_index()
)


In [15]:
top_categories = (
    user_interest.sort_values(["user_id", "score"], ascending=[True, False])
    .groupby("user_id")
    .head(3)
)

In [16]:
recommended_places = plac.merge(top_categories, on="category", how="inner")
recommended_places = recommended_places[["user_id", "place_id", "name", "city", "category"]]

In [17]:
recommended_places = recommended_places.merge(user_interest, on=["user_id","category"], how="left")
recommended_places = recommended_places.sort_values(["user_id", "score"], ascending=[True, False])

top5_places = recommended_places.groupby("user_id").head(5).reset_index(drop=True)



In [18]:
print(top5_places)

      user_id  place_id                                  name        city  \
0           1         4           Red Beach Resort of Nuweiba     Nuweiba   
1           1         5               Port Said Diving Center   Port Said   
2           1         7            White Island of Alexandria  Alexandria   
3           1        10                       Siwa Beach Club        Siwa   
4           1        22                     Marsa Alam Island  Marsa Alam   
...       ...       ...                                   ...         ...   
1245      250         2               Siwa Rock Climbing Site        Siwa   
1246      250         8                 Port Said Hiking Path   Port Said   
1247      250        11  Golden Water Sports Center of Fayoum      Fayoum   
1248      250        14                 Fayoum Mountain Trail      Fayoum   
1249      250        15                        Cairo Zip Line       Cairo   

       category  score  
0         beach     21  
1         beach     21  
